## Retial Analytics Juptyer Notebook Starter Pack
### Sellers' Reward System Management Dashboard and Performance Prediction System POC for Zimla (Ecommerce Marketplace Company)

### How to access Kaggle dataset 


##### Note to use Kaggle API
1. you would need to create a Kaggle account. Sign up here
2. retrieve to authentication credential after logging into Kaggle visit your <strong>Account</strong> page, scroll down to the API section and click on 'Create New API Token'
3. this would download a kaggle.json file on your PC. You are required to move this json file to the folder location 'C:\Users<Windows-username>.kaggle\' so that you have full file path that looks like this 'C:\Users<Windows-username>.kaggle\kaggle.json'
4. completing step 1 to 3 would be adequate for the this project

refer to the [Kaggle API documentation](https://www.kaggle.com/docs/api#getting-started-installation-&-authentication) for more details 

In [1]:
# package to access kaggle api
!pip install kaggle

In [2]:
# Download the  Olist dataset
import kaggle
kaggle.api.dataset_download_files('olistbr/brazilian-ecommerce', path='data/', unzip=True)

### Import relevant Python packages

In [3]:
import pandas as pd
import numpy as np

### Dataset Details:  
Details on the Dataset can be found here - [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/olistbr/brazilian-ecommerce)

### Dataset Relationship Diagram


![title](https://i.imgur.com/HRhd2Y0.png)

#### Load dataset

In [4]:
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
order = pd.read_csv('data/olist_orders_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')

product_category_translation =  pd.read_csv('data/product_category_name_translation.csv')  #English translation for the products

#### Merge the data files possibly to one big file

In [5]:
order1 = pd.merge(order_items, order_payments, how = 'left', on = 'order_id')
order2 = pd.merge(order1, order_reviews, how = 'left', on = 'order_id')
order3 = pd.merge(order2, order, how = 'left', on = 'order_id')
data1 = pd.merge(order3, products, how = 'left', on = 'product_id')
data2 = pd.merge(data1, sellers, how = 'left', on = 'seller_id')
hugedata = pd.merge(data2, customers, how = 'left', on = 'customer_id')
hugedata.shape

(118318, 39)

In [6]:
hugedata.dtypes

order_id                          object
order_item_id                      int64
product_id                        object
seller_id                         object
shipping_limit_date               object
price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      object
payment_installments             float64
payment_value                    float64
review_id                         object
review_score                       int64
review_comment_title              object
review_comment_message            object
review_creation_date              object
review_answer_timestamp           object
customer_id                       object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
product_category

#### Converting the Unique Identifiers to Simple Ones (Oder_id, Product_id, and Seller_id)b

In [7]:
#Creating simpler unique for the order id 
temp_id_index1 = (np.arange(hugedata.order_id.nunique())+1)
new_order_id = np.char.add('order_',temp_id_index1.astype('str'))
#print(new_order_id)

# Merging the Old order id and the newly generated order id using dict
temp_order_id_dict = dict(zip(hugedata.order_id.unique(), new_order_id))
print()
#print(list(temp_order_id_dict.items())[:5])

# Replacing the order id with the new order id
hugedata['new_order_id'] = hugedata['order_id'].map(temp_order_id_dict)

In [8]:
#Creating simpler unique for the product id 
temp_id_index2 = (np.arange(hugedata.product_id.nunique())+1)
new_product_id = np.char.add('product_',temp_id_index2.astype('str'))
#print(new_product_id)

# Merging the old product id and the newly generated product id using dict
temp_product_id_dict = dict(zip(hugedata.product_id.unique(), new_product_id))
print()
#print(list(temp_product_id_dict.items())[:5])

# Replacing the product id with the new product id
hugedata['new_product_id'] = hugedata['product_id'].map(temp_product_id_dict)

['product_1' 'product_2' 'product_3' ... 'product_32949' 'product_32950'
 'product_32951']

[('4244733e06e7ecb4970a6e2683c13e61', 'product_1'), ('e5f2d52b802189ee658865ca93d83a8f', 'product_2'), ('c777355d18b72b67abbeef9df44fd0fd', 'product_3'), ('7634da152a4610f1595efa32f14722fc', 'product_4'), ('ac6c3623068f30de03045865e4e10089', 'product_5')]


In [9]:
#Creating simpler unique for the seller id 
temp_id_index3 = (np.arange(hugedata.seller_id.nunique())+1)
new_seller_id = np.char.add('seller_',temp_id_index3.astype('str'))
#print(new_seller_id)

# Merging the old seller id and the newly generated seller id using dict
temp_seller_id_dict = dict(zip(hugedata.seller_id.unique(), new_seller_id))
print()
#print(list(temp_seller_id_dict.items())[:5])

# Replacing the seller id with the new seller id
hugedata['new_seller_id'] = hugedata['seller_id'].map(temp_seller_id_dict)

In [10]:
#Creating simpler unique for the customer id 
temp_id_index4 = (np.arange(hugedata.customer_id.nunique())+1)
new_customer_id = np.char.add('customer_',temp_id_index4.astype('str'))
#print(new_customer_id)

# Merging the old customer id and the newly generated customer id using dict
temp_customer_id_dict = dict(zip(hugedata.customer_id.unique(), new_customer_id))
print()
#print(list(temp_customer_id_dict.items())[:5])

# Replacing the seller id with the new seller id
hugedata['new_customer_id'] = hugedata['customer_id'].map(temp_customer_id_dict)

In [12]:
hugedata.isnull().sum()

order_id                              0
order_item_id                         0
product_id                            0
seller_id                             0
shipping_limit_date                   0
price                                 0
freight_value                         0
payment_sequential                    3
payment_type                          3
payment_installments                  3
payment_value                         3
review_id                             0
review_score                          0
review_comment_title             104229
review_comment_message            67650
review_creation_date                  0
review_answer_timestamp               0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                    15
order_delivered_carrier_date       1254
order_delivered_customer_date      2588
order_estimated_delivery_date         0
product_category_name              1709


In [17]:
cols = ['order_id','product_id','seller_id', 'review_comment_title','review_comment_message','review_creation_date',
        'product_name_lenght','product_length_cm', 'product_height_cm', 'product_width_cm']    
hugedata.drop(cols, axis= 1, inplace=True)

KeyError: "['order_id' 'product_id' 'seller_id' 'review_comment_title'\n 'review_comment_message' 'review_creation_date' 'product_name_lenght'\n 'product_length_cm' 'product_height_cm' 'product_width_cm'] not found in axis"

In [14]:
#Save the Refined dataset as the new dataset
#hugedata.to_csv('refineddata.csv')

In [16]:
## Order purchase Dataset
orders_purchase = hugedata.loc[:,['new_seller_id', 'new_order_id', 'new_product_id', 'order_item_id',
                                'price', 'order_status', 'order_purchase_timestamp']]
orders_purchase.head()

,new_seller_id,new_order_id,new_product_id,order_item_id,price,order_status,order_purchase_timestamp
118313,seller_742,order_98662,product_1728,1,299.99,delivered,2018-04-23 13:57:06
118314,seller_1581,order_98663,product_27138,1,350.00,delivered,2018-07-14 10:26:46
118315,seller_49,order_98664,product_13743,1,99.90,delivered,2017-10-23 17:07:56
118316,seller_913,order_98665,product_10335,1,55.99,delivered,2017-08-14 23:02:59
118317,seller_192,order_98666,product_32951,1,43.00,delivered,2018-06-09 17:00:18


,order_item_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,...,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,new_order_id,new_product_id,new_seller_id,new_customer_id
0,1,2017-09-19 09:45:35,58.90,13.29,1.0,credit_card,2.0,72.19,97ca439bc427b48bc1cd7177abe71365,5,...,volta redonda,SP,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,order_1,product_1,seller_1,customer_1
1,1,2017-05-03 11:05:13,239.90,19.93,1.0,credit_card,3.0,259.83,7b07bacd811c4117b742569b04ce3580,4,...,sao paulo,SP,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,order_2,product_2,seller_2,customer_2
2,1,2018-01-18 14:48:30,199.00,17.87,1.0,credit_card,5.0,216.87,0c5b33dea94867d1ac402749e5438e8b,5,...,borda da mata,MG,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,order_3,product_3,seller_3,customer_3
3,1,2018-08-15 10:10:18,12.99,12.79,1.0,credit_card,2.0,25.78,f4028d019cb58564807486a6aaf33817,4,...,franca,SP,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,order_4,product_4,seller_4,customer_4
4,1,2017-02-13 13:57:51,199.90,18.14,1.0,credit_card,3.0,218.04,940144190dcba6351888cafa43f3a3a5,5,...,loanda,PR,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,order_5,product_5,seller_5,customer_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,2018-07-18 04:25:09,232.75,28.58,1.0,boleto,1.0,261.33,f58325a189b533108ea7141e53ee15ba,3,...,votorantim,SP,7e765e16bdd8cf6bb0dc7728e1294636,45960,medeiros neto,BA,order_84,product_85,seller_78,customer_84
96,1,2018-08-23 15:11:40,48.90,18.44,1.0,credit_card,1.0,67.34,2bf3890219fda8d54c35c8b70253fb7f,5,...,rio de janeiro,RJ,93b592cbb89abbda7dc554c39add344e,13848,mogi-guacu,SP,order_85,product_86,seller_79,customer_85
97,1,2017-08-23 01:25:39,87.00,12.11,1.0,credit_card,8.0,99.11,2a74b0559eb58fc1ff842ecc999594cb,5,...,ibitinga,SP,d56ded4569dd9821b2f83710a56afbde,4173,sao paulo,SP,order_86,product_87,seller_13,customer_86
98,1,2017-08-23 01:25:39,87.00,12.11,1.0,credit_card,8.0,99.11,89a02c45c340aeeb1354a24e7d4b2c1e,5,...,ibitinga,SP,d56ded4569dd9821b2f83710a56afbde,4173,sao paulo,SP,order_86,product_87,seller_13,customer_86
